In [ ]:
# %%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install -q tensorflow-text
!pip install annoy
!pip install simpleneighbors[annoy]
# !pip install -q nltk
# !pip install -q tqdm

In [1]:
import json
import nltk
import os
import pprint
import simpleneighbors
import urllib
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm


import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

In [43]:

nltk.download('punkt')
questions = []
# Create the global index object
index = simpleneighbors.SimpleNeighbors(512, metric='angular')

def load_data(path='../data/faq.csv', sep="##,##"):
    print('Loading data....')
    data = pd.read_csv(path,sep='##,##')
    print('...... done')
    return data

def extract_sentences_from_answer(df):
  all_sentences = []
  for index, row in df.iterrows():
      # nltk.tokenize.sent_tokenize is a unsupervised algo
      sentences = nltk.tokenize.sent_tokenize(row['answer'])    
      sentences = sentences + nltk.tokenize.sent_tokenize(row['question'])
      all_sentences.extend(zip(sentences, [row['answer']] * len(sentences)))
  return list(set(all_sentences)) # remove duplicates

def extract_questions(df):  
  for index, row in df.iterrows():
    questions.append((row['question'], row['answer']))   
  return list(set(questions))
        

def get_nearest(query_text):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)
  return search_results

def load_USE_model(path='/home/sandipan/projects/model/'):  
  print('Loading Universal Sentence Encoder from', path)
  m = hub.load(path)
  print('Universal Sentence Encoder model from in mem', m)
  return m

def build_search_index(data, model, batch_size = 10, index_path='/home/sandipan/projects/index/faq.ann'):
  sentences = extract_sentences_from_answer(data)
  encodings = model.signatures['response_encoder'](
    input=tf.constant([sentences[0][0]]),
    context=tf.constant([sentences[0][1]]))
  print('Computing embeddings for %s sentences' % len(sentences))
  slices = zip(*(iter(sentences),) * batch_size)
  num_batches = int(len(sentences) / batch_size)
  print('Batch wise index add...')
  for s in tqdm(slices, total=num_batches):
    response_batch = list([r for r, c in s])
    context_batch = list([c for r, c in s])
    encodings = model.signatures['response_encoder'](
                  input=tf.constant(response_batch),
                  context=tf.constant(context_batch)
                )    
    for batch_index, batch in enumerate(response_batch):
      index.add_one(batch, encodings['outputs'][batch_index])

  print('Building Index...')
  index.build()
  print('Saving Index...')
  index.save(index_path)
  return dict(sentences)




[nltk_data] Downloading package punkt to /home/sandipan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
data = load_data('../data/faq.csv')
print(data.head(2))
model = load_USE_model('/home/sandipan/projects/model/')
# extract_questions(data)
sentence_dict = build_search_index(data = data,model = model, batch_size=10)

Loading data....
...... done
                   question                                             answer
0            What is NEFT ?  National Electronic Funds Transfer or NEFT is ...
1  Where is mount everest ?  Mount Everest is the highest of the Himalayan ...
Loading Universal Sentence Encoder from /home/sandipan/projects/model/
Universal Sentence Encoder model from in mem <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f36bf4d2410>
Computing embeddings for 39 sentences
Batch wise index add...



Building Index...
Saving Index...


In [73]:
results = get_nearest('NEFT')
ans = pd.DataFrame(columns=['original_q','answer','confidence'])    
for result in results:
    print(result, ans[ans['answer'] == sentence_dict[result]].size)
    if(ans[ans['answer'] == sentence_dict[result]].size == 0):
        values = ['',sentence_dict[result],100]
        zipped = zip(ans.columns, values)    
        a_dictionary = dict(zipped)
        ans = ans.append(a_dictionary,ignore_index=True)  
ans

What is NEFT ? 0
National Electronic Funds Transfer or NEFT is an electronic funds transfer system maintained by the Reserve Bank of India. 3
Started in November 2005, the setup was established and maintained by Institute for Development and Research in Banking Technology 3
what is the price of sari ? 0
When I use Popmoney when will the recipient receive the funds? 0
If you are still unable to activate your card, please call us at 800-868-4262 or 919-420-8000. 0
It also offers functionality not found on the desktop site, such as Mobile Check Deposit and Tap Balances. 0
The Digital Banking mobile app contains most of the same features as the desktop site. 3
Often times, it can take up to 3 days after accepting the payment. 3
If a fee is imposed, it will be deducted from the amount of the incoming wire prior to us receiving it and posting it to your account. 0


,original_q,answer,confidence
0,,National Electronic Funds Transfer or NEFT is ...,100
1,,The pink one cost about 250,100
2,,This depends on the delivery option selected b...,100
3,,"Please remember to hit the pound key (""#"") aft...",100
4,,The Digital Banking mobile app contains most o...,100
5,,While Coastal does not charge a fee for receiv...,100


In [69]:
ans[ans['answer'] == 'The pinkss one cost about 250'].size

0

In [70]:
anss = pd.DataFrame(columns=['original_q','answer','confidence']) 
anss[anss['answer'] == 'The pinkss one cost about 250'].size  

0